In [5]:
"""
Algorithm in order to move a group of units in a strategy game:

Here we consider tanks and factories (inspired by https://www.youtube.com/watch?v=8kBQMQyLHME):
- each turn, you can move all your tanks
- every (N / number of factories) turns, you can pop a new tank at one of your factories
- the goal is to eliminate the adversary (no tank, no factories left)
"""

from collections import defaultdict, deque
from dataclasses import dataclass
import enum
import numpy as np
from typing import *

%matplotlib inline
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio
from PIL import Image
from IPython.display import HTML
import seaborn

In [10]:
"""
Game mechanics
"""


Position = Tuple[int, int]


class Move(enum.Enum):
    NONE = 0
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4
    
    @classmethod
    def all(cls):
        return [cls.NONE, cls.LEFT, cls.RIGHT, cls.UP, cls.DOWN]
    
    
class Player(enum.Enum):
    PLAYER_1 = 0
    PLAYER_2 = 1
    
    def next_player(self):
        return self.PLAYER_1 if self == self.PLAYER_2 else self.PLAYER_2


@dataclass(frozen=True)
class Territory:
    width: int
    height: int
        
    def displace(self, position: Position, move: Move):
        x, y = position
        if move == Move.LEFT:
            x -= 1
        elif move == Move.RIGHT:
            x += 1
        elif move == Move.UP:
            y += 1
        elif move == Move.DOWN:
            y -= 1
        return (x, y) if ((0 <= x < self.width) and (0 <= y < self.height)) else position

    
class TankPositions:
    def __init__(self):
        self.positions = defaultdict(int)
    
    def add(self, position: Position):
        self.positions[position] += 1
    
    def remove(self, position: Position):
        count = self.positions.get(position, 0)
        if count == 0:
            raise "No tank at position " + str(position)
        elif count == 1:
            del self.positions[position]
        else:
            self.positions[position] = count - 1
    
    def __contains__(self, position):
        return position in self.positions
    
    def __repr__(self):
        return repr(self.positions)
    
    def items(self):
        return self.positions.items()


@dataclass(frozen=False)
class PlayerPieces:
    factories: Set[Position]
    tanks: TankPositions
    
    def pop_positions(self, territory: Territory):
        pop_positions = {territory.displace(factory, move) for factory in self.factories for move in Move.all()}
        pop_positions -= {factory for factory in self.factories}
        return pop_positions
    
    @classmethod
    def init_pieces(cls, territory: Territory, player: Player, nb_factories: int):
        factories = set()
        for y in np.random.choice(range(territory.height), size=nb_factories, replace=False):
            factories.add((cls.start_column(territory, player), y))
        return cls(tanks=TankPositions(), factories=factories)
    
    @staticmethod
    def start_column(territory: Territory, player: Player):
        return 0 if player == Player.PLAYER_1 else territory.width-1

        
@dataclass(frozen=False)
class GameState:
    territory: Territory
    players: Dict[Player, PlayerPieces]
    
    @classmethod
    def init_game(cls, width: int, height: int, nb_factories: int):
        territory = Territory(width, height)
        return GameState(
            territory = territory,
            players = {
                Player.PLAYER_1: PlayerPieces.init_pieces(territory, Player.PLAYER_1, nb_factories),
                Player.PLAYER_2: PlayerPieces.init_pieces(territory, Player.PLAYER_2, nb_factories)
            })


@dataclass(frozen=True)
class Observation:
    territory: Territory
    current_player: PlayerPieces
    opponent_player: PlayerPieces


@dataclass(frozen=True)
class AvailableActions:
    tanks: TankPositions()
    pop_locations: Set[Position]

    
@dataclass(frozen=False)
class Turn:
    tank_moves: List[Tuple[Position, Move]]
    pop_location: Position
        
    @classmethod
    def init(cls):
        return cls(tank_moves=[], pop_location=None)
    
    
class TankWars:    
    def __init__(self, game_state: GameState):
        self.current_player = Player.PLAYER_1
        self.game_state = game_state
    
    def get_player_state(self, player: Player) -> PlayerPieces:
        return self.game_state.players[player]
    
    def get_winner(self) -> Player:
        if not self.game_state.players[Player.PLAYER_1].factories:
            return Player.PLAYER_2
        if not self.game_state.players[Player.PLAYER_2].factories:
            return Player.PLAYER_1
        return None
    
    def get_observation(self) -> Observation:
        return Observation(
            territory=self.game_state.territory,
            current_player=self._current_player_state(),
            opponent_player=self._opponent_player_state()
        )
    
    def get_actions(self) -> AvailableActions:
        player = self._current_player_state()
        pop_locations = player.pop_positions(self._get_territory())
        return AvailableActions(tanks=player.tanks, pop_locations=pop_locations)
    
    def play(self, turn: Turn) -> bool:
        player = self._current_player_state()
        available_actions = self.get_actions()
        self._play_tanks(player, available_actions, turn)
        if turn.pop_location:
            if turn.pop_location in available_actions.pop_locations:
                player.tanks.add(turn.pop_location)
            else:
                raise "Invalid pop location"
        self.current_player = self.current_player.next_player()
        return self.get_winner() is not None
    
    def _play_tanks(self, player: PlayerPieces, available_actions: AvailableActions, turn: Turn):
        other_player = self._opponent_player_state()
        for curr_pos, move in turn.tank_moves:
            if curr_pos not in available_actions.tanks:
                raise "Invalid move: tank does not exist"
            next_pos = self.game_state.territory.displace(curr_pos, move)
            if next_pos in player.factories:
                raise "Invalid move: tank cannot go over own factory"
            if next_pos in other_player.tanks:
                other_player.tanks.remove(next_pos)
            else:
                player.tanks.add(next_pos)
                if next_pos in other_player.factories:
                    other_player.factories.remove(next_pos)
            player.tanks.remove(curr_pos)
    
    def _current_player_state(self) -> PlayerPieces:
        return self.game_state.players[self.current_player]
    
    def _opponent_player_state(self) -> PlayerPieces:
        return self.game_state.players[self.current_player.next_player()]
    
    def _get_territory(self) -> Territory:
        return self.game_state.territory


"""
Random movement based AI
"""


def choose(choices):
    choices = list(choices)
    return choices[np.random.randint(0, len(choices))]


def random_ia_play(game: TankWars):
    observation = game.get_observation()
    available_actions = game.get_actions()
    turn = Turn.init()
    turn.pop_location = choose(available_actions.pop_locations)
    for curr_pos, count in available_actions.tanks.items():
        valid_moves = []
        for move in Move.all():
            next_pos = observation.territory.displace(curr_pos, move)
            if next_pos not in observation.current_player.factories:
                valid_moves.append(move)
        move = np.random.choice(valid_moves)
        turn.tank_moves.extend((curr_pos, move) for _ in range(count))
    return turn

        
"""
Diffusion algorithm based AI
"""


def neighbor_positions(territory: Territory, position: Position):
    x, y = position
    if x > 0:
        yield x-1, y
    if x < territory.width - 1:
        yield x+1, y
    if y > 0:
        yield x, y-1
    if y < territory.height - 1:
        yield x, y+1
        

def diffusion_map(territory: Territory, positions: List[Tuple[Position, float]], alpha: float):
    m = np.zeros((territory.width, territory.height))
    to_visit = deque(positions)
    while to_visit:
        position, value = to_visit.popleft()
        m[position[0]][position[1]] = value    # For the init part...
        for x, y in neighbor_positions(territory, position):
            if m[x][y] == 0:
                m[x][y] = value * alpha
                to_visit.append(((x, y), value * alpha))
    return m


def diffusion_ia_play(game: TankWars):
    observation = game.get_observation()
    territory = observation.territory
    player = observation.current_player
    other = observation.opponent_player
    available_actions = game.get_actions()
    
    enemy_tanks = diffusion_map(territory, [(tank, count * 10.) for tank, count in other.tanks.items()], alpha=0.8)
    enemy_factories = diffusion_map(territory, [(factory, 25.) for factory in other.factories], alpha=0.8)
    allied_tanks = diffusion_map(territory, [(tank, count * 10.) for tank, count in player.tanks.items()], alpha=0.8)
    tank_map = enemy_tanks + enemy_factories
    pop_map = enemy_tanks - allied_tanks
    
    turn = Turn.init()
    
    best_pop = float('-inf')
    for x, y in available_actions.pop_locations:
        if pop_map[x][y] > best_pop:
            best_pop = pop_map[x][y]
            turn.pop_location = (x, y)
    
    for curr_pos, count in available_actions.tanks.items():
        best_move = None
        best_val = float('-inf')
        for move in Move.all():
            x, y = territory.displace(curr_pos, move)
            if (x, y) not in player.factories:
                if tank_map[x][y] > best_val:
                    best_val = tank_map[x][y]
                    best_move = move
        turn.tank_moves.extend((curr_pos, best_move) for _ in range(count))
    
    return turn

In [11]:
game_state = GameState.init_game(width=15, height=10, nb_factories=3)
game = TankWars(game_state)


def player_on_map(territory: Territory, player: PlayerPieces):
    m = np.zeros((territory.width, territory.height))
    for (x, y), count in player.tanks.items():
        m[x][territory.height - y - 1] = count
    for factory in player.factories:
        x, y = factory
        m[x][territory.height - y - 1] = -1
    return np.transpose(m) # used for display


def get_tank_pattern():
    return np.array([
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]
    ])


def get_factory_pattern():
    return np.array([
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.]
    ])


def render(image, tank_pattern, factory_pattern):
    assert factory_pattern.shape == tank_pattern.shape
    init_h, init_w = image.shape
    scale_h, scale_w = tank_pattern.shape
    h = init_h * scale_h
    w = init_w * scale_w
    magnified = np.zeros((h, w))
    for i in range(init_h):
        for j in range(init_w):
            x = scale_h * i
            y = scale_w * j
            if image[i][j] >= 1:
                magnified[x:x+scale_h,y:y+scale_w] = tank_pattern
            elif image[i][j] < 0:
                magnified[x:x+scale_h,y:y+scale_w] = factory_pattern
    return magnified


# https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python
def to_image(game: TankWars):
    territory = game.get_observation().territory
    tank_pattern = get_tank_pattern()
    factory_pattern = get_factory_pattern()
    image = np.zeros((territory.height * tank_pattern.shape[0], territory.width  * tank_pattern.shape[1], 3), 'uint8')
    image[..., 0] = render(player_on_map(territory, game.get_player_state(Player.PLAYER_1)), tank_pattern, factory_pattern) * 255
    image[..., 2] = render(player_on_map(territory, game.get_player_state(Player.PLAYER_2)), tank_pattern, factory_pattern) * 255
    image = Image.fromarray(image, 'RGB')
    image = image.resize((300, 200))
    return image


def debug(nb_turns: int, as_gif: bool, on_screen: bool):
    if on_screen:
        fig, ax = plt.subplots(figsize=(5, 5 * nb_turns), nrows=nb_turns)
        #frames = []
    
    images = [to_image(game)]
    for i in range(nb_turns):
        if as_gif:
            images.append(to_image(game))
        if on_screen:
            frame = ax[i].imshow(to_image(game), animated=True)
            frames.append([frame])
            
        turn = diffusion_ia_play(game) if i % 2 == 0 else random_ia_play(game)
        winner = game.play(turn)
        if winner:
            print("Player", game.get_winner(), "wins")
            break

    if as_gif:
        imageio.mimsave('diffusion_2.gif', images)
    if on_screen:
        # ani = animation.ArtistAnimation(fig, frames, interval=50, repeat_delay=1000)
        plt.show()

    
debug(nb_turns=1000, on_screen=False, as_gif=True)
print("Done...")

Player Player.PLAYER_1 wins
Done...


In [ ]:
# Main problem with the diffusion algorithm is that several tanks stacked will always behave the same (unless you tank into account each tank move)... and that tank will always move !
# TODO - you can also introduce a bit of random (ponderated move by the map around the tank)